In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.layers import Dense
from keras.models import Model, Sequential


from TCGAMultiOmics.multiomics import MultiOmicsData
from moge.network.heterogeneous_network import HeterogeneousNetwork


Using TensorFlow backend.


#  Import network from file

In [2]:
import pickle

# WRITE
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'wb') as file:
#     pickle.dump(network, file)

# READ
with open('moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_network_new.pickle', 'rb') as file:
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_biogrid.pickle', 'rb') as file:
    network = pickle.load(file)
#     network.remove_extra_nodes()
#     network.node_list = network.all_nodes
#     node_list = network.node_list

In [3]:
for u,v,d in network.G.edges(data=True):
    if d["type"] == 'u_n':
        d['weight']+=1e-8

In [19]:
# READ edgelists
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_test_edges.pickle', 'rb') as file:
    test_edges_dict = pickle.load(file)
    
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_val_edges.pickle', 'rb') as file:
    val_edges_dict = pickle.load(file)

# Load training data

In [20]:
# X, y = network.multi_omics_data.load_data(modalities=["MIR", "GE"])

In [21]:
# network.multi_omics_data.external_data_path = "/home/jonny/PycharmProjects/Bioinformatics_ExternalData/"

In [22]:
# X["MIR"].shape

(460, 1870)

# Training Source Target Graph Embedding

In [4]:
from keras.layers import Input, Conv1D, Lambda, Dot, Dense, Flatten, MaxPooling1D, Lambda
from keras.layers import LSTM, Dense, TimeDistributed, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K

from keras.optimizers import SGD, Adam, RMSprop
from keras.losses import binary_crossentropy

from keras.utils import to_categorical


def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = np.random.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=np.random.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)


In [53]:
K.clear_session()
tf.reset_default_graph()
# sess.close()
sess = tf.InteractiveSession()

/home/jonny/.conda/envs/jonny_conda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [54]:
# node_features_size = X["MIR"].shape[0]
input_shape = (None, 4)
_d = 128
batch_size = 1

In [55]:
# with tf.name_scope('inputs'):
E_ij = Input(batch_shape=(batch_size, 1), name="E_ij")
#     input_i = tf.placeholder(tf.float32, shape=input_shape, name="input_i")
#     input_j = tf.placeholder(tf.float32, shape=input_shape, name="input_j")
input_seq_i = Input(batch_shape=(batch_size, None, 4), name="input_seq_i")
input_seq_j = Input(batch_shape=(batch_size, None, 4), name="input_seq_j")
# is_directed = tf.placeholder(tf.bool, name="is_directed")
is_directed = Input(batch_shape=(batch_size, 1), dtype=tf.bool, name="is_directed")
# i = tf.Variable(int, name="i", trainable=False)
# j = tf.Variable(int, name="j", trainable=False)

In [56]:
def create_base_network(input_shape):
    """ Base network to be shared (eq. to feature extraction).
    """
    input = Input(shape=input_shape)
#     x = Flatten()(input)
    x = LSTM(128, input_shape=input_shape, return_sequences=False)(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

def source_target_emebedding_distance(vects):
    emb_i, emb_j, is_directed = vects
    dot_directed = Dot(axes=1)([emb_i[:, 0:int(_d/2)], emb_j[:, int(_d/2):_d]])
    dot_undirected = Dot(axes=1)([emb_i, emb_j])
    return K.switch(is_directed, K.sigmoid(dot_directed), K.sigmoid(dot_undirected))

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [57]:
#build create_base_network to use in each siamese 'leg'
lstm_network = create_base_network(input_shape=(None, 4))

print("lstm_network", lstm_network)

# encode each of the two inputs into a vector with the convnet
encoded_i = lstm_network(input_seq_i)
encoded_j = lstm_network(input_seq_j)
print("encoded_i", encoded_i, "\nencoded_j", encoded_j)

distance = Lambda(source_target_emebedding_distance)([encoded_i, encoded_j, is_directed])
print("distance", distance)

siamese_net = Model(inputs=[input_seq_i, input_seq_j, is_directed], outputs=distance)

lstm_network <keras.engine.training.Model object at 0x7fc2fe43ff98>
encoded_i Tensor("model_1/dense_3/Relu:0", shape=(1, 128), dtype=float32) 
encoded_j Tensor("model_1_1/dense_3/Relu:0", shape=(1, 128), dtype=float32)
distance Tensor("lambda_1/Select_1:0", shape=(1, 1), dtype=float32)


In [58]:
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss=contrastive_loss, 
                    optimizer=RMSprop(lr=0.01),
                    metrics=[accuracy])

siamese_net.count_params()


117632

# Data Generator

In [11]:
from moge.network.data_generator import DataGenerator

In [12]:
generator = DataGenerator(network.node_list, network=network, 
                          batch_size=1, dim=(None, 4), shuffle=True)

Ed_count 448908 Eu_count 2550360 En_count 2307016


In [34]:
network.multi_omics_data.LNC.get_genes_info().iloc[2]["Transcript sequence"]

'TGAAAACTTCCTGAGGCCTCCTCAGAAGCAGATGCTGCTATGCTTCCCGTACAGCCTGAAGAACCAAACATTTCTATACATTTATGAGACCCAACTCCAAAAGCATACTGGAATGGAATTAAAGACCGAACTACAGAAGCTAAAGAGAGCTGATCAAACTAAAAAAAAAAGTTAAAGGTGAGGAGGCCAAGGCTCAGAAGAGTCACTTGCCCATGATCACTGCTCTTATAGCGCCAACTCAGAGCTCAGGCAATGCTCATGTTCTTTCCACTGTGCCTCATCGCTCATGTCTGACTCTTCTAGAAATGTGGGCAAATCCCCTGCCTTCTGTGGGTCTCAGATTTCCATAAAAAATAAAATCAATGGATCAACTTAA'

In [41]:
X, y = generator.__getitem__(0)

In [44]:
X["input_seq_i"].shape

(1, 2595, 4)

In [38]:
%timeit generator.seq_to_array(_34)

401 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Train

In [ ]:
siamese_net.fit_generator(generator)

Epoch 1/1
    877/5306284 [..............................] - ETA: 1568:13:57 - loss: 0.7648 - accuracy: 0.0000e+00